In [66]:
import warnings
from statsbombpy.api_client import NoAuthWarning
warnings.simplefilter('ignore', NoAuthWarning)
from socceraction.data.statsbomb import StatsBombLoader

api = StatsBombLoader(getter="remote", creds={"user": "", "passwd": ""})

In [67]:
import random

df_competitions = api.competitions()
random_row = df_competitions.sample(1)

competition_id = random_row['competition_id'].item()
season_id = random_row['season_id'].item()
competition_name = random_row['competition_name'].item()
season_name = random_row['season_name'].item()

df_games = api.games(competition_id=competition_id, season_id=season_id)

all_teams = list(df_games['home_team_id'].unique())
team_id = random.choice(all_teams)

df_games = df_games[(df_games['away_team_id'] == team_id) | (df_games['home_team_id'] == team_id)]
all_games = list(df_games['game_id'])
g = random.choice(all_games)

c_stage = df_games[df_games['game_id'] == g]['competition_stage'].item()
df_teams = api.teams(game_id=g)
df_players = api.players(game_id=g)
df_events = api.events(game_id=g)
team_name = df_events[df_events['team_id'] == team_id]['team_name'].iloc[0]
teams = list(df_events['team_name'].unique())
teams.remove(team_name)

c_string = f"{team_name} vs {teams[0]} ({competition_name} {c_stage} - {season_name})"
print(c_string)

West Ham United LFC vs Reading WFC (FA Women's Super League Regular Season - 2019/2020)


In [73]:
import pandas as pd
from tabulate import tabulate
import mplsoccer
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe

df_team = df_events[df_events['team_id'] == team_id].copy()
index_first_sub = df_team[df_team.type_name == "Substitution"].index.min()
df_events_pre_sub = df_team[df_team.index < index_first_sub]
df_passes = df_events_pre_sub[df_events_pre_sub.type_name == "Pass"]
df_received = df_passes[df_passes['extra'].apply(lambda x: 'pass' in x and 'outcome' in x['pass']) == False].reset_index(drop=True)
df_received['receiver'] = df_received['extra'].apply(lambda x: x.get('pass', {}).get('recipient', {}).get('name'))
df_received['location_end'] = df_received['extra'].apply(lambda x: x.get('pass', {}).get('end_location'))


df = df_received.copy()

df[['x', 'y']] = pd.DataFrame(df['location'].tolist())
df[['x_end', 'y_end']] = pd.DataFrame(df['location_end'].tolist())

df = df[['player_name', 'x', 'y', 'receiver', 'x_end', 'y_end']]

player_pass_count = df.groupby('player_name').size().to_frame("num_passes")

df['pair_key'] = df.apply(lambda x: "_".join(sorted([x['player_name'], x['receiver']])), axis = 1)
pair_pass_count = df.groupby('pair_key').size().to_frame("num_passes")
pair_pass_count = pair_pass_count[pair_pass_count['num_passes'] > 3]
player_position = df.groupby('player_name').agg({"x" : "mean", "y" : "mean"})

fig, ax = plt.subplots(figsize=(20, 36), dpi = 600)
pitch = mplsoccer.Pitch(pitch_type = 'statsbomb')
pitch.draw(ax)

max_player_count = player_pass_count.num_passes.max()
max_pair_count = pair_pass_count.num_passes.max()

# ax.scatter(player_position['x'], player_position['y'], color='red', edgecolor='black')

for pair_key, row in pair_pass_count.iterrows():
    player1, player2 = pair_key.split("_")

    player1_x = player_position.loc[player1]['x']
    player1_y = player_position.loc[player1]['y']

    player2_x = player_position.loc[player2]['x']
    player2_y = player_position.loc[player2]['y']

    num_passes = row["num_passes"]

    line_width = num_passes/max_pair_count

    ax.plot([player1_x, player2_x], [player1_y, player2_y], 'w-', linestyle='-', alpha = 0.4, lw = 5*line_width, zorder = 3, color = 'purple')


for player_name, row in player_pass_count.iterrows():
    player_x = player_position.loc[player_name]['x']
    player_y = player_position.loc[player_name]['y']

    num_passes = row["num_passes"]

    marker_size = num_passes/max_player_count

    ax.plot(player_x, player_y, '.', markersize = 200*marker_size, color = 'purple', zorder=5)
    ax.plot(player_x, player_y, '.', markersize = 200*marker_size - 30, color = 'white', zorder = 6)
    ax.annotate(player_name.split()[-1], xy=(player_x, player_y), ha='center', va='center', zorder=7, weight='bold', size = 15, path_effects=[pe.withStroke(linewidth = 2, foreground = 'white')])

